## Indicador de Ausencia: variable para capturar los valores nulos NA

En los notebooks anteriores aprendimos cómo reemplazar los valores ausentes por la media, mediana, moda o extrayendo una muestra aleatoria. Estos métodos asumen que los datos ausentes faltan al azar (MCAR).

Hay otros métodos que pueden ser usados cuando los valores no están ausentes de forma aleatoria, por ejemplo la sustitución por un valor arbitrario o un valor al final de la distribución. Sin embargo, estas técnicas de sustitución pueden afectar la distribución de las variables dramáticamente, y por lo tanto no ser apropiadas para usar con modelos lineales.

**Qué podemos hacer si los datos no son MCAR y queremos usar un modelo lineal?**

Si los datos no faltan de forma aleatoria, es una buena idea reemplazar las observaciones con la media/mediana/moda y una variable binaria, o en otras palabras, un **Indicador de Ausencia**. El indicador de ausencia toma el valor 1 si el dato falta, o 0 si está presente.


### Para cuáles variables podemos añadir un indicador de ausencia?

Podemos añadir  un indicador de ausencia a variables numéricas y categóricas.

### Nota

Nunca se debe añadir un indicador de ausencia solo, por el contrario, siempre es usado junto con otras técnicas de imputación, como la sustitución por la media/mediana para variables numéricas y la sustitución por categoría frecuente para variables categóricas. También podemos usar la sustitución por muestra aleatoria con un indicador de ausencia para ambos tipos de variables.

Usualmente se usan en conjunto:

- Sustitución por la media/mediana + indicador de ausencia (Variables numéricas)
- Sustitución por categoría frecuente + indicador de ausencia (Variables categóricas)
- Sustitución por muestra aleatoria  + indicador de ausencia (Variables numéricas y categóricas)

### Condiciones

- Los datos no están ausentes de forma aleatoria
- Los datos ausentes tienen poder predictivo

### Ventajas

- Fácil de implementar
- Captura la importancia de los datos ausentes si existe

### Limitaciones

- Expande el espacio de las variables
- La variable original todavía necesita ser sustituida para remover los valores nulos

Añadir un indicador de ausencia incrementa 1 variable por cada variable en el conjunto de datos que tengan datos ausentes. Por ejemplo, si un conjunto de datos tiene 10 variables, y todas ellas tienen valores ausentes, después de añadir un indicador de ausencia tendremos 20 variables: las 10 variables originales más las 10 variables binarias adicionales que indican si el valor estaba ausente o no. Este puede no ser un problema en conjuntos de datos con unos cientos de variables,  pero si tenemos miles de variables, crear una variable adicional para indicar si hay valores nulos, incrementará considerablemente el tamaño de nuestros datos.

### Importante

Adicionalmente, la misma observación puede tener datos faltantes en múltiples variables, lo cual lleva a que los indicadores de ausencia sean muy similares o de hechos idénticos entre ellos.

### Nota Final

Típicamente la sustitución por la media/mediana/moda se hace en conjunto con el agregado de un indicador de ausencia. De este modo se cubren dos aspectos: si los datos estaban ausentes de forma completamente aleatoria, se captura con la sustitución por la media/mediana/moda y si este no era el caso, se captura con el indicador de ausencia.

Ambos métodos son extremadamente sencillos de implementar y por lo tanto la opción favorita en las competencias de ciencia de datos. Por ejemplo, miren la solución ganadora de la competencia KDD 2009: ["Winning the KDD Cup Orange Challenge with Ensemble Selection](http://www.mtome.com/Publications/CiML/CiML-v3-book.pdf).


## En este demo:

Usaremos ambos conjuntos de datos: House Price y Titanic.

- Para bajar los datos, por favor referirse a la clase en **Datasets** en la  **Sección 1** del curso.


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# separar los datasets
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [2]:
# carguemos los datos con las columnas seleccionadas

data = pd.read_csv('../titanic.csv', usecols=['age', 'fare', 'survived'])
data.head()

,survived,age,fare
0,1,29.0000,211.3375
1,1,0.9167,151.5500
2,0,2.0000,151.5500
3,0,30.0000,151.5500
4,0,25.0000,151.5500


In [3]:
# evaluaemos el porcentaje de valores nulos
data.isnull().mean()

survived    0.000000
age         0.200917
fare        0.000764
dtype: float64

Para añadir un indicador de ausencia binario, no necesitamos 'aprender' nada del set de entrenamiento, por lo tanto en principio, podemos hacer esto en el conjunto de datos original y luego separar los datos. Sin embargo, no recomendamos hacer esto.

Adicionalmente, si usas Scikit-learn, o Feature-engine para añadir el indicador de ausencia, de la forma que han sido diseñados, necesitan aprender del set de enteramiento cuales variables deben.

Más adelante vamos a explorar las diferentes implementaciones para este método, por ahora, vamos a crear el indicador binario manualmente.


In [4]:
# separar datos en sets entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(
    data[['age', 'fare']],  # variables
    data['survived'],  # target
    test_size=0.3,  # porcentaje de obs en test set
    random_state=0)  # semilla

X_train.shape, X_test.shape

((916, 2), (393, 2))

In [5]:
# Exploremos los valores ausentes en el set de entrenamiento

X_train.isnull().mean()

age     0.191048
fare    0.000000
dtype: float64

In [6]:
# añadir el indicador de ausencia

# esto se puede hacer de forma sencilla usando np.where de numpy
# y isnull de pandas:

X_train['age_NA'] = np.where(X_train['age'].isnull(), 1, 0)
X_test['age_NA'] = np.where(X_test['age'].isnull(), 1, 0)

X_train.head()

,age,fare,age_NA
501,13.0,19.5000,0
588,4.0,23.0000,0
402,30.0,13.8583,0
1193,NaN,7.7250,1
686,22.0,7.7250,0


In [7]:
# la media de la variable binaria, coincide con el porcentaje 
# de valores ausentes en la variable original

X_train['age_NA'].mean()

0.19104803493449782

In [8]:
# sin embargo, la variable original todavía tiene valores faltantes
# los cuales necesitan ser reemplazados por cualquiera de las 
# técnicas que aprendimos

X_train.isnull().mean()

age       0.191048
fare      0.000000
age_NA    0.000000
dtype: float64

In [9]:
# por ejemplo la sustitución por la media
median = X_train['age'].median()

X_train['age'] = X_train['age'].fillna(median)
X_test['age'] = X_test['age'].fillna(median)

# revisemos que ya no existen valores faltantes
X_train.isnull().mean()

age       0.0
fare      0.0
age_NA    0.0
dtype: float64

### House Prices dataset

Repitamos el ejercicio con el set de datos de House Prices.

In [10]:
# vamos a usar las siguientes variables,
# unas son numéricas y otras categóricas

cols_to_use = [
    'LotFrontage', 'MasVnrArea', # numericas
    'BsmtQual', 'FireplaceQu', # categorica
    'SalePrice' # target
]

In [11]:
# carguemos los datos de House Prices 

data = pd.read_csv('../houseprice.csv', usecols=cols_to_use)
print(data.shape)
data.head()

(1460, 5)


,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,SalePrice
0,65.0,196.0,Gd,NaN,208500
1,80.0,0.0,Gd,TA,181500
2,68.0,162.0,Gd,TA,223500
3,60.0,0.0,TA,Gd,140000
4,84.0,350.0,Gd,TA,250000


In [12]:
# explorems los valores ausentes

data.isnull().mean()

LotFrontage    0.177397
MasVnrArea     0.005479
BsmtQual       0.025342
FireplaceQu    0.472603
SalePrice      0.000000
dtype: float64

In [13]:
# separamos los datos para entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(data,
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

In [14]:
# creemos una función para añadir un indicador de ausencia
# en forma de variable binaria

def missing_indicator(df, variable):    
    return np.where(df[variable].isnull(), 1, 0)

In [15]:
# agreguemos un indicador binario
# con la función que creamos

for variable in cols_to_use:
    X_train[variable+'_NA'] = missing_indicator(X_train, variable)
    X_test[variable+'_NA'] = missing_indicator(X_test, variable)
    
X_train.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,SalePrice,LotFrontage_NA,MasVnrArea_NA,BsmtQual_NA,FireplaceQu_NA,SalePrice_NA
64,NaN,573.0,Gd,NaN,219500,1,0,0,1,0
682,NaN,0.0,Gd,Gd,173000,1,0,0,0,0
960,50.0,0.0,TA,NaN,116500,0,0,0,1,0
1384,60.0,0.0,TA,NaN,105000,0,0,0,1,0
1100,60.0,0.0,TA,NaN,60000,0,0,0,1,0


In [16]:
# ahora evaluemos el valor promedio de los indicadores de ausencia

# primero capturemos el indicador de ausencia con un 
# list comprehension
missing_ind = [col for col in X_train.columns if 'NA' in col]

# calculemos el promedio
X_train[missing_ind].mean()

LotFrontage_NA    0.184932
MasVnrArea_NA     0.004892
BsmtQual_NA       0.023483
FireplaceQu_NA    0.467710
SalePrice_NA      0.000000
dtype: float64

In [17]:
# el promedio del indicador de ausencia coincide
# con el porcentaje de valores ausentes
# en la variable original

X_train.isnull().mean()

LotFrontage       0.184932
MasVnrArea        0.004892
BsmtQual          0.023483
FireplaceQu       0.467710
SalePrice         0.000000
LotFrontage_NA    0.000000
MasVnrArea_NA     0.000000
BsmtQual_NA       0.000000
FireplaceQu_NA    0.000000
SalePrice_NA      0.000000
dtype: float64

In [18]:
# creemos una función para llenar los valores ausentes con un valor:
# vamos a usar una función que usamos en los notebooks anteriores
# con la cual ya estarás familiarizado

def impute_na(df, variable, value):
    return df[variable].fillna(value)

In [19]:
# sustituimos los valores nulos con la mediana para las variables numéricas
# recuerda que calculamos la mediana usando el set de entrenamiento

median = X_train['LotFrontage'].median()
X_train['LotFrontage'] = impute_na(X_train, 'LotFrontage', median)
X_test['LotFrontage'] = impute_na(X_test, 'LotFrontage', median)

median = X_train['MasVnrArea'].median()
X_train['MasVnrArea'] = impute_na(X_train, 'MasVnrArea', median)
X_test['MasVnrArea'] = impute_na(X_test, 'MasVnrArea', median)


# ahora sustituyamos los valores nulos de las variables categóricas con la
# categoría más frecuente (moda)
# la moda debe ser calculada sobre el set de entrenamiento
mode = X_train['BsmtQual'].mode()[0]
X_train['BsmtQual'] = impute_na(X_train, 'BsmtQual', mode)
X_test['BsmtQual'] = impute_na(X_test, 'BsmtQual', mode)

mode = X_train['FireplaceQu'].mode()[0]
X_train['FireplaceQu'] = impute_na(X_train, 'FireplaceQu', mode)
X_test['FireplaceQu'] = impute_na(X_test, 'FireplaceQu', mode)

In [20]:
# revisemos que no hay mas valores nulos
X_train.isnull().mean()

LotFrontage       0.0
MasVnrArea        0.0
BsmtQual          0.0
FireplaceQu       0.0
SalePrice         0.0
LotFrontage_NA    0.0
MasVnrArea_NA     0.0
BsmtQual_NA       0.0
FireplaceQu_NA    0.0
SalePrice_NA      0.0
dtype: float64

Como pueden ver, ahora tenemos el doble de variables que en los datos originales: empezamos con 4 variables, y terminamos con 8, más el target.

**Es todo por este demo. Esperemos lo hayan disfrutado.
Nos vemos en el siguiente!! **
